In [ ]:
# Basic NGSolve things
from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw

import matplotlib.pyplot as plt
import numpy as np
from read_gmsh3D import ReadGmsh
mesh = Mesh(ReadGmsh('data/field/geometry/gmsh.msh'))
import compare_my_method
import matplotlib.pyplot as plt
plt.rc('text', usetex=False)


mesh.ngmesh.SetBCName(0, "left")
mesh.ngmesh.SetBCName(1, "right")
mesh.ngmesh.SetBCName(2, "front")
mesh.ngmesh.SetBCName(3, "back")
for j in range(52):
    mesh.ngmesh.SetBCName(j+6, "frac")
# print(mesh.GetBoundaries(), len(mesh.GetBoundaries()))
    

gfu = GridFunction(H1(mesh))
gfu.Set(1, definedon=mesh.Boundaries("inflow"))
Draw(gfu)
V = Compress(SurfaceL2(mesh, order=0, definedon=mesh.Boundaries("frac")))
print("matrix elements: ", mesh.ne, "; fracture elements: ", V.ndof)

# 2b: Hybrid-Mixed

In [ ]:
K0, epsK1 = 1, 100

order = 0
frac = "frac"
V = Discontinuous(HDiv(mesh, order=order, RT=True))
W = L2(mesh, order=order)
M = FacetFESpace(mesh, order=order)
Vf = Discontinuous(HDivSurface(mesh, order=order, RT=True), BND=True)
for el in Vf.Elements(BND):
    if el.mat =="frac":
        for dof in el.dofs:
            Vf.FreeDofs()[dof] = True
            Vf.SetCouplingType(dof, COUPLING_TYPE.INTERFACE_DOF)
    else:
        for dof in el.dofs:
            Vf.FreeDofs()[dof] = False
            Vf.SetCouplingType(dof, COUPLING_TYPE.UNUSED_DOF)
Vf = Compress(Vf)

Mf = Compress(HCurl(mesh, order=order, definedon=mesh.Boundaries(frac))) # hacker?!

fes = V*W*M*Vf*Mf


h = specialcf.mesh_size
n = specialcf.normal(3)
t = specialcf.tangential(3)
con = Cross(n,t) # co-normal
t0 = specialcf.tangential(3,True)

(u, p, phat, uf, pbar0), (v, q, qhat, vf, qbar0) = fes.TnT()
pbar = pbar0.Trace()*t0
qbar = qbar0.Trace()*t0

condense=True
a = BilinearForm(fes, condense=condense)
helperx0 = IfPos((x+200)*(x+500), 0, 1)
helpery0 = IfPos((y-1200)*(y-1500), 0, 1)
helpery1 = IfPos((y-100)*(y-400), 0, 1)
helperz0 = IfPos((z-300)*(z-500), 0, 1)
helperz1 = IfPos((z-100)*(z+100), 0, 1)

# subdomain 1
a += (1/K0*u*v-p*div(v)-q*div(u))*dx
a += (1/epsK1*uf.Trace()*vf.Trace()-phat.Trace()*div(vf).Trace()-qhat.Trace()*div(uf).Trace()
     )*ds("frac")
a += (v*n*phat+u*n*qhat)*dx(element_boundary=True)
a += (vf.Trace()*con*pbar+uf.Trace()*con*qbar)*ds(element_boundary=True, definedon=mesh.Boundaries("frac"))
# dirichlet bc hacker
a += helpery1*helperz1*1e8*phat.Trace()*qhat.Trace()*ds("left|right")

f = LinearForm(fes)
f += -helperx0*helperz0*qhat.Trace()*ds("back")
f += -helpery0*helperz0*qhat.Trace()*ds("left")
gfu = GridFunction(fes)
a.Assemble()
f.Assemble()

if condense==True:
    f.vec.data += a.harmonic_extension_trans * f.vec
gfu.vec.data = a.mat.Inverse(fes.FreeDofs(condense), inverse="pardiso")*f.vec
if condense==True:
    gfu.vec.data += a.harmonic_extension * gfu.vec 
    gfu.vec.data += a.inner_solve * f.vec

# Local postprocessing
V2 = L2(mesh, order=order+1, all_dofs_together=False)
ph = GridFunction(V2)

a2 = BilinearForm(V2)
f2 = LinearForm(V2)

p2, q2 = V2.TnT()
a2 += grad(p2)*grad(q2)*dx
a2.Assemble()
f2 += -1/K0*gfu.components[0]*grad(q2)*dx
f2.Assemble()

V2.FreeDofs()[:mesh.ne]=False
ph.vec[:mesh.ne].data=gfu.components[1].vec[:mesh.ne]
ph.vec.data += a2.mat.Inverse(V2.FreeDofs())*f2.vec

gfInterp = GridFunction(H1(mesh, order=order+1))
gfInterp.Set(ph)
Draw(gfInterp, mesh, "soln")

ng = sum(fes.FreeDofs(True))
nt = sum(fes.FreeDofs())
print("NE: ", mesh.ne, "NDOF-g: ", ng, "NDOF-M:", sum(M.FreeDofs(True)), 
      "NDOF-Vf: ", sum(Vf.FreeDofs(True)), "NDOF-Mf: ", sum(Mf.FreeDofs(True)), "NDOF-l:", nt-ng)

# data visualization over line
xx = np.linspace(0, 1, 101)
h = np.array([ph(mesh(350-850*x, 100+1400*x, -100+600*x)) for x in xx])
h2 = np.array([ph(mesh(-500+850*x, 100+1400*x, -100+600*x)) for x in xx])
xx *= sqrt(850**2+1400**2+600**2)

plt.figure(figsize=(10,8), dpi=400)
case = 4
plot_id = "a"
ref_index = 0
subdomain_id = 0
compare_my_method.evaluate(xx, h, case, plot_id, ref_index, subdomain_id, name="HM-DFM")
plt.axis([-50,1800, -20, 720])
plt.savefig("data/fieldA.pdf", dpi=400,bbox_inches='tight')


plt.figure(figsize=(10,8), dpi=400)
case = 4
plot_id = "a"
ref_index = 0
subdomain_id = 1
plt.axis([-50,1800, -10, 270])
compare_my_method.evaluate(xx, h2, case, plot_id, ref_index, subdomain_id, name="HM-DFM")
plt.savefig("data/fieldB.pdf", dpi=400,bbox_inches='tight')